# Demo: Serenity Scenario API

Serenity lets you run stress tests on linear products -- portfolios of digital asset tokens -- in its current version. You can run scenarios that you create yourself, called custom scenarios, or you can use canned or predefined scenarios packaged with the Serenity platform. The API lets you manage your custom scenarios, execute scenario runs and retrieve run results programmatically, so if you wish to develop your own suite of regular scenarios to run against your portfolio you can easily script them.

In [1]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [2]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [3]:
import datetime
import re

from datetime import date
from uuid import UUID, uuid4

import pandas as pd

# create an alias to the api
api = connect_widget.get_api()

## Getting started

To run a scenario the first thing we need is a portfolio:

In [4]:
from serenity_types.portfolio.core import SimplePortfolio

asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

# scenarios uses a special type, SimplePortfolio -- wrap it
simple_portfolio = SimplePortfolio(portfolio_id=uuid4(),
                                   portfolio_name='Test Portfolio',
                                   portfolio_manager='Test PM',
                                   base_currency_id=uuid4(),
                                   asset_positions=portfolio.to_asset_positions())

The starting point is to acquire a Scenarios API wrapper:

In [5]:
scenarios = api.scenarios()

## Listing custom scenarios

In [6]:
custom_scenarios = scenarios.get_custom_scenarios().result
rows = []
for scenario in custom_scenarios:
    rows.append({'scenario_id': scenario.scenario_id, 'scenario_name': scenario.name})
pd.DataFrame(rows)

,scenario_id,scenario_name
0,1c1fdcd1-052c-4273-a3cc-410917512b53,Custom Scenario - 2022-10-21 09:10:38.985755
1,42413ee4-16ad-41a6-b409-637347660eec,Updated Scenario Name - 2022-10-21 09:10:39.53...
2,7128cc34-ae85-44a0-b6d4-6b679ecc7995,Custom Scenario - 2022-10-21 09:10:40.317501
3,0a56707d-ce96-49ab-9fcc-6c32a5adb8d1,Updated Scenario Name - 2022-10-21 09:10:41.85...
4,137c4bb9-33af-4f6e-a347-71d75de82f22,Updated Scenario Name - 2022-10-21 09:10:42.63...
...,...,...
149,79b2fd41-53a9-42d9-ba09-d2f3d3033e40,Custom Scenario - 2022-12-15 06:34:58.213353
150,12d5d59a-7d12-43a4-b85e-4bfe695a8644,Custom Scenario - 2022-12-15 06:34:58.726150
151,6da9c98a-9ed9-438d-b7a7-a31d780bb890,FTX Bankruptcy
152,520b088f-5c19-4e13-9e71-5d44f5579f58,ZJ Scenario


## Listing predefined scenarios

In [7]:
predefined_scenarios = scenarios.get_predefined_scenarios().result
scenario_by_name = {}
rows = []
for scenario in predefined_scenarios:
    name_elements = re.split(':\s', scenario.name)
    model_config_id = UUID(name_elements[0])
    scenario_name = name_elements[1]
    rows.append({'scenario_id': scenario.scenario_id, 'model_config_id': model_config_id, 'scenario_name': scenario_name})
    scenario_by_name[scenario_name] = {'scenario': scenario, 'model_config_id': model_config_id}
pd.DataFrame(rows)

,scenario_id,model_config_id,scenario_name
0,d983f302-a3d4-4d35-a713-768ac8a9ff68,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,Facebook Renaming to Meta
1,4e98301b-b901-4c2e-adf9-f8d5f38310c4,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,LUNA Crash and Crypto Credit Crunch
2,5ebf5855-38c2-4be6-9301-13fe1c30d6d7,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,3AC Insolvency
3,dbf00795-f5b6-48ed-9093-fac7f76ad7af,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,Wormhole Bridge Hack
4,e7c69342-ae7d-4e31-a7aa-340e67db1cbd,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,Ethereum Merge Announcement
5,8ad3c606-8229-4af5-b898-ba57c4603486,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,2017 Bull Market
6,6f33ec34-c363-4f26-80a9-e3bd26adc757,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,2018 Bear Market - First Crypto Winter
7,619db52d-074b-4c48-ae17-d3995f6804fc,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,2019 Bull Market
8,b4b7c5a9-ffd1-4b63-a9fa-40936f8f5670,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,2020 COVID Crash
9,ffcf07dd-0919-4734-b9e0-fc5902771c28,ce3a7d83-4133-404a-99ac-5f3777f8dc2d,2019 COVID Downturns and Sideways Market - Sec...


## Managing custom scenarios

In [8]:
from serenity_types.risk.scenarios import ScenarioCloneRequest, ScenarioDefinition, ScenarioSource

# create a custom scenario, letting the server allocated ID and version
custom = ScenarioDefinition(scenario_id=None, scenario_version=None,
                            source=ScenarioSource.CUSTOM,
                            name='Demo Custom Scenario',
                            shocks=[], last_updated_by='SDK notebook')
custom = scenarios.create_custom_scenario(custom).result

# update it
custom.name='Demo Custom Scenario MODIFIED'
custom = scenarios.update_custom_scenario(custom).result

# rollback
custom = scenarios.rollback_custom_scenario(custom.scenario_id, version=custom.scenario_version).result

# clone it
clone_request = ScenarioCloneRequest(scenario_id=custom.scenario_id, scenario_name=f'{custom.name} - CLONED')
custom_clone = scenarios.clone_scenario(clone_request).result

# delete the scenario and its clone
assert scenarios.delete_custom_scenario(custom.scenario_id).result.deleted
assert scenarios.delete_custom_scenario(custom_clone.scenario_id).result.deleted

## Running a predefined scenario

In [9]:
from serenity_sdk.client.raw import SerenityError
from serenity_types.risk.scenarios import ScenarioRequest
from serenity_types.pricing.core import PricingContext

scenario_info = scenario_by_name['3AC Insolvency']
predefined_scenario = scenario_info['scenario']
model_config_id = scenario_info['model_config_id']
start_date = date(2022, 12, 26)
pricing_context = PricingContext(as_of_date=start_date)
request = ScenarioRequest(scenario_id=None, scenario=predefined_scenario, portfolio=simple_portfolio,
                          pricing_context=pricing_context, model_config_id=model_config_id,
                          start_date=start_date, end_date=start_date, schema_version=1)

scenarios.run_scenario(request)

Response(request_id=UUID('46a6c6af-f9a5-4ff6-9e46-a7b5a8cbb742'), as_of_date=datetime.date(2022, 12, 26), warnings=[], result=ScenarioResult(portfolio_pnl=PnL(base_pnl=76233.18064846103, total_shock_pnl=203607.46473071218, pnl_by_shock=[ShockPnL(shock_id=UUID('1bcca942-b993-4358-8119-6f10c7505801'), shock_pnl=-4042.9920372105225), ShockPnL(shock_id=UUID('1ff71aab-4d9a-43da-b7c6-09d2ac8265a2'), shock_pnl=1537.7288060782555), ShockPnL(shock_id=UUID('3d2e4587-1599-43a5-9d66-a66ba45fc843'), shock_pnl=11819.44847895085), ShockPnL(shock_id=UUID('52b3433f-62b7-43c7-9ef6-1e7698be902f'), shock_pnl=-29338.132872336202), ShockPnL(shock_id=UUID('56f0096b-d011-476d-b01a-38cd528d3d97'), shock_pnl=16134.73895441492), ShockPnL(shock_id=UUID('770dd6f9-72fb-4d9b-9f29-21b3e3bf6081'), shock_pnl=-29665.089355515287), ShockPnL(shock_id=UUID('8b9dbfda-a714-4e46-af95-3517c17569ff'), shock_pnl=12758.799090638639), ShockPnL(shock_id=UUID('96e58bde-6088-4c0b-ba88-30a8a2bd7537'), shock_pnl=25276.84574416292), Sho